In [1]:
import os
import gc
import sys
import argparse
import time
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms.functional as TF
from torch.utils.data import DataLoader
from torchvision import utils
from torchvision.models.detection.rpn import AnchorGenerator
from PIL import Image
from transforms import Resize, ToTensor, Compose
from data.coco import Coco
from data.kitti_dataset import KITTI
from data.voc import VOC
from backbone.backbone_vgg import vgg16, vgg11, vgg13, vgg19
from backbone.backbone_resnet import resnet18
from models.faster_rcnn_mod import FasterRCNN
from models.mask_rcnn_mod import MaskRCNN
from models.keypoint_rcnn_mod import KeypointRCNN
from models.retinanet_mod import RetinaNet
from engine import train_one_epoch, evaluate
from matplotlib.patches import Rectangle

ModuleNotFoundError: No module named 'pycocotools'

In [ ]:
transform = []
transform.append(Resize((300, 1200)))
transform.append(ToTensor())
transform = Compose(transform)

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

def draw_box(box, label, score=''):
    x1, y1, x2, y2 = box
    color = plt.cm.get_cmap('tab10')(label)
    rect = Rectangle((x1,y1),x2-x1,y2-y1,linewidth=2,
                     edgecolor='k',
                     facecolor=color,alpha=0.5)
    plt.gca().add_patch(rect)
    

def plot_img(image, target=None, thld=0.5):
    image = image.permute(1,2,0).numpy()
    plt.imshow(image)
    if target:
        for i in range(len(target['labels'])):
            scores = target.get('scores')
            if scores is not None:
                if scores[i]<thld:
                    continue
                draw_box(target['boxes'][i], int(target['labels'][i]), str(float(scores[i]))[:4])
            else:
                draw_box(target['boxes'][i], int(target['labels'][i]))

In [ ]:
KITTI_CLASSES = ['BG', 'Car', 'Van', 'Truck', 'Pedestrian',
                'Person_sitting', 'Cyclist', 'Tram', 'Misc', 'DontCare']


class KITTI(object):
    def __init__(self, path, transforms=None):
        self.path = path
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(os.path.join(path, "training/image_2"))))
        self.labels = list(sorted(os.listdir(os.path.join(path, "training/label_2"))))

    def __getitem__(self, idx):
        img_path = os.path.join(self.path, "training/image_2", self.imgs[idx])
        label_path = os.path.join(self.path, "training/label_2", self.labels[idx])
        image = Image.open(img_path).convert("RGB")
        
        boxes, label = [], []
        objs = 0
        for i in open(label_path).readlines():
            i = i.split()
            label.append(KITTI_CLASSES.index(i[0]))
            boxes.append(list(map(float, i[4:8])))
            objs += 1
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        label = torch.as_tensor(label, dtype=torch.int64)
        img_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:,2] - boxes[:, 0])
        iscrowd = torch.zeros((objs, ), dtype=torch.int64)
        
                  
        target = {}
        target["boxes"] = boxes
        target["labels"] = label
        target["image_id"] = img_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        
        if self.transforms is not None:
            image, target = self.transforms(image, target)
        
        return image, target

    def __len__(self):
        return len(self.imgs)

    


In [ ]:
root = 'D:/Dataset/KITTI'
data = KITTI(root, transform)
num_classes = 10
indices = torch.randperm(len(data)).tolist()
dataset = torch.utils.data.Subset(data, indices[:7000])
testdata = torch.utils.data.Subset(data, indices[7000:])
traindata = DataLoader(dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)
data_loader_test = DataLoader(testdata, batch_size=1, shuffle=True, collate_fn=collate_fn)

In [ ]:
root = '/home/ajay/Desktop/ID_CNN/Dataset/COCO/train2017/'
annotations = '/home/ajay/Desktop/ID_CNN/Dataset/COCO/annotations/instances_train2017.json'
data = Coco(root = root, annotations=annotations, transforms=transform)
indices = torch.randperm(len(data)).tolist()
dataset = torch.utils.data.Subset(data, indices[:110000])
testdata = torch.utils.data.Subset(data, indices[110000:])
traindata = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
data_loader_test = DataLoader(testdata, batch_size=2, shuffle=True, collate_fn=collate_fn)    

In [ ]:
root = '/home/ajay/Desktop/ID_CNN/Dataset/VOCdevkit/VOC2012'
data = VOC(root, transform)
num_classes = 20
indices = torch.randperm(len(data)).tolist()
dataset = torch.utils.data.Subset(data, indices[:16000])
testdata = torch.utils.data.Subset(data, indices[16000:])
traindata = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
data_loader_test = DataLoader(testdata, batch_size=2, shuffle=True, collate_fn=collate_fn)

In [ ]:
img, trgt = dataset[100]
print(trgt)
plt.figure(figsize=(20,20))
plot_img(img, trgt)

In [ ]:
bbox = [2, 4, 6, 8]